# XOX PnF charts
## Development Notes

Two methods for populating charts:

- **High and Low** price: whenever H and L are available
- **Last/Close** price: for illiquid assets (or indices). Also for intraday trading using realtime data

Scale types:

- Constant size boxes
- Variable size boxes

#### Chart creation process:

1. Loading data into dataframe - **get_price_data()**
2. getting the scale for the chart - **generate_scale()**
3. generate a table with rows representing the boxes drawn by a line of data. Each row has a *trend status* (either 1 or -1), a column low and a column high. Those values represent *boxes*, and must be included in the chart scale. This is handled by **get_pnf_ranges()**, which calls **init_pnf()**
  * call  **init_pnf()** first
  * **update_pnf()**
  
```
get_chart() -> get_price_data()
               generate_scale()
               get_pnf_ranges() -> init_pnf()
                                   update_pnf()
               get_pnf_changes()
               get_pnf_columns() -> generate_column_range()
```

### Preparations

In [1]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.2
numpy version: 1.22.3


In [2]:
# inputs:
data_file = 'EXP1.csv'
reversal_size = 3
box_size = 1

In [4]:
chart_params = {
    'data_file': data_file,
    'reversal_size': reversal_size,
    'box_size': box_size
}

In [74]:
from xox_pnf.pnfplot import * 

Functions from pnfplot
---------------------------

In [19]:
price_data = get_price_data(data_file)

In [20]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2001-01-01 to 2001-01-05
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    5 non-null      float64
 1   Low     5 non-null      float64
 2   Close   5 non-null      float64
dtypes: float64(3)
memory usage: 160.0 bytes


In [21]:
scale = generate_scale(start=int(np.floor(price_data['Low'].min())),
                    end=int(np.ceil(price_data['High'].max())), box_size=box_size)

print("scale array: ", scale)

scale array:  [35 36 37 38 39 40 41]


### The next two steps are managed by get_pnf_ranges()

In [ ]:
# initialise status and box arrays:
trend_status = np.zeros(len(price_data))
box_low = np.zeros(len(price_data))
box_high = np.zeros(len(price_data))

# Initialise the chart until a trend status (+/-1) is found
box_range = []
for index, row in enumerate(price_data.iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = init_pnf(scale, high, low, close, reversal_size, box_range)
    print("step ", index,"status " ,status,"box range", box_range)
    trend_status[index] = status
    box_low[index] = box_range.min()
    box_high[index] = box_range.max()
    if status != 0:
        break

In [ ]:
print(trend_status, box_range)

In [ ]:
# update_pnf()

### get_pnf_ranges() and get_pnf_changes()

In [38]:
# This includes a call to init_pnf() as in the previous cell
pnf_data = get_pnf_ranges(price_data, scale, reversal_size)
pnf_data

,trend_status,range_low,range_high
0,1.0,36.0,38.0
1,1.0,36.0,40.0
2,-1.0,37.0,39.0
3,-1.0,37.0,39.0
4,1.0,38.0,40.0


In [39]:
# Detects change of trend and adds it to time frame
# Note that the change column is 'shifted': it's True when a status change is detected on the next price line

pnf_data = get_pnf_changes(pnf_data)
pnf_data

,trend_status,range_low,range_high,change
0,1.0,36.0,38.0,False
1,1.0,36.0,40.0,True
2,-1.0,37.0,39.0,False
3,-1.0,37.0,39.0,True
4,1.0,38.0,40.0,True


### Generating the final column arrays

In [40]:
# The final chart columns: pairs of trend status and array of boxes
columns = get_pnf_columns(pnf_data, scale)
columns

[(1.0, array([36, 37, 38, 39, 40])),
 (-1.0, array([37, 38, 39])),
 (1.0, array([38, 39, 40]))]

In [41]:
# The whole process is managed by get_chart():

# get_chart()

## Printing the PnF chart

In [42]:
# Using the pnf_text() function to generate pnf as text

print(pnf_text(scale,columns))

41.......41
40..X.X..40
39..XOX..39
38..XOX..38
37..XO...37
36..X....36
35.......35


In [44]:
type(pnf_text(scale,columns))

str

## Building the pnf_chart class

In [75]:
class pnf_chart():
    
    def __init__(self, chart_params):
        data_file = chart_params['data_file']
        
        self.reversal_size = chart_params['reversal_size']
        self.box_size = chart_params['box_size']
        self.price_data = get_price_data(data_file)
        
        start = int(np.floor(self.price_data['Low'].min()))
        end = end=int(np.ceil(self.price_data['High'].max()))
        self.scale = generate_scale(start, end, self.box_size)
        
        pnf_data = get_pnf_ranges(price_data, scale, reversal_size)
        self.pnf_data = get_pnf_changes(pnf_data)
        
        self.columns = get_pnf_columns(pnf_data, scale)
        
    def __str__(self):
        return pnf_text(self.scale, self.columns)
    
    def __repr__(self):
        return f'PnF chart of {data_file}'

In [76]:
pnf = pnf_chart(chart_params)

In [77]:
pnf.pnf_data

,trend_status,range_low,range_high,change
0,1.0,36.0,38.0,False
1,1.0,36.0,40.0,True
2,-1.0,37.0,39.0,False
3,-1.0,37.0,39.0,True
4,1.0,38.0,40.0,True


In [78]:
pnf.columns

[(1.0, array([36, 37, 38, 39, 40])),
 (-1.0, array([37, 38, 39])),
 (1.0, array([38, 39, 40]))]

In [79]:
pnf

PnF chart of EXP1.csv

In [80]:
print(pnf)

41.......41
40..X.X..40
39..XOX..39
38..XOX..38
37..XO...37
36..X....36
35.......35
